In [10]:
def scraper():

    #Libraries and dependencies
    import pandas as pd
    from bs4 import BeautifulSoup as bs
    import requests
    from splinter import Browser, redirect
    from webdriver_manager.chrome import ChromeDriverManager

    mars_info = {}

    ######################################################
    ### Scrap Mars News Website 

    #set splinter and prepare soup
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    url = 'https://redplanetscience.com/'
    browser.visit(url)
    html = browser.html
    soup = bs(html, 'html.parser')
    browser.quit()

    #extract data
    dates = soup.find_all('div', class_= 'list_date')
    titles = soup.find_all('div', class_= 'content_title')
    paragraphs = soup.find_all('div', class_= 'list_date')

    #save data
    news= []
    for i in range(0, len(dates)):
        news.append({
            'date': dates[i].text,
            'title': titles[i].text,
            'paragraph': paragraphs[i].text
        })
    
    mars_info['news'] = news

    ######################################################
    ### Scrap Mars High Res Image

    #Spinter, Navigate and prepare soup
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    url = 'https://spaceimages-mars.com/'
    browser.visit(url)
    browser.links.find_by_partial_text('FULL IMAGE').click()
    html = browser.html
    soup = bs(html, 'html.parser')
    browser.quit()

    #find data
    featured_image_url = soup.find_all('img', class_= 'fancybox-image')
    featured_image_url = 'https://spaceimages-mars.com/'+featured_image_url[0]['src']

    #save data
    mars_info['feature_img'] = featured_image_url

    ######################################################
    ### Scrap Mars Facts

    #get tables and format
    url = 'https://galaxyfacts-mars.com/'
    tables = pd.read_html(url)
    fact_table = tables[0]
    fact_table.columns = ['Comparison', 'Mars', 'Earth']
    fact_table = fact_table.iloc[1:len(fact_table),:]
    fact_table = fact_table.set_index('Comparison')
    fact_table = fact_table.to_dict()

    #save data
    mars_info['fact_table'] = fact_table

    ######################################################
    ### Scrap Mars Hemisphere Imagaes

    # Setup splinter, navigate and retreive data
    executable_path = {'executable_path': ChromeDriverManager().install()}
    browser = Browser('chrome', **executable_path, headless=False)
    url = 'https://marshemispheres.com/'
    browser.visit(url)
    hemisphere_image_urls = []

    for i in range(0, 4):
        links_found = browser.links.find_by_partial_text('Hemisphere Enhanced')
        links_found[i].click()
        html = browser.html
        soup = bs(html, 'html.parser')
    
        #retreive data
        title = soup.find('h2', class_= 'title').text
        img_url = soup.find('div', class_= 'downloads')
        img_url = img_url.find('a', target= '_blank')['href']
        img_url = 'https://marshemispheres.com/' + img_url
    
        hemisphere_image_urls.append({
            'Title': title,
            'Image URL': img_url
        })
    
        browser.links.find_by_partial_text('Back').click()

    browser.quit()

    #save data
    mars_info['hemisphere_imgs'] = hemisphere_image_urls

    ######################################################
    
    return mars_info

In [11]:
mars_info = scraper()
mars_info



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/martinsingla/.wdm/drivers/chromedriver/mac64/91.0.4472.19/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/martinsingla/.wdm/drivers/chromedriver/mac64/91.0.4472.19/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/martinsingla/.wdm/drivers/chromedriver/mac64/91.0.4472.19/chromedriver] found in cache


{'news': [{'date': 'June 7, 2021',
   'title': "NASA's Perseverance Rover Will Peer Beneath Mars' Surface ",
   'paragraph': 'June 7, 2021'},
  {'date': 'June 6, 2021',
   'title': 'Mars Is Getting a New Robotic Meteorologist',
   'paragraph': 'June 6, 2021'},
  {'date': 'June 4, 2021',
   'title': '7 Things to Know About the Mars 2020 Perseverance Rover Mission',
   'paragraph': 'June 4, 2021'},
  {'date': 'May 29, 2021',
   'title': "Curiosity Mars Rover's Summer Road Trip Has Begun",
   'paragraph': 'May 29, 2021'},
  {'date': 'May 28, 2021',
   'title': "A Martian Roundtrip: NASA's Perseverance Rover Sample Tubes",
   'paragraph': 'May 28, 2021'},
  {'date': 'May 28, 2021',
   'title': "NASA's Perseverance Mars Rover Gets Balanced",
   'paragraph': 'May 28, 2021'},
  {'date': 'May 27, 2021',
   'title': "NASA's Mars 2020 Rover Closer to Getting Its Name",
   'paragraph': 'May 27, 2021'},
  {'date': 'May 27, 2021',
   'title': 'MAVEN Maps Electric Currents around Mars that are Funda

In [12]:
import pymongo
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)
db = client.mars_db
scrap_data = db.scrap_data
scrap_data.update({}, mars_info, upsert=True)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  


{'n': 1,
 'nModified': 0,
 'upserted': ObjectId('60bd99d957c007b2c15597ff'),
 'ok': 1.0,
 'updatedExisting': False}